In [610]:
import math as math
import cntk as c
import numpy as np
import matplotlib.pyplot as plt
c.cntk_py.set_fixed_random_seed(1)
np.random.seed(1)

In [611]:
def cntk_erf(x):
    return c.tanh((2 / math.sqrt(math.pi)) * x)

#x ~ Norm(Mean, Var)
#Prob(x<b) = 0.5* (1 + erf((b-Mean)/(sqrt(2*Var))))
#Prob(a<x<b) = 0.5 * erf((b-Mean)/(sqrt(2*Var)))] - 0.5 * erf((a-Mean)/(sqrt(2*Var)))]
def cntk_normal_cdf(x, mean, variance):
    #return 0.5 * (1 + cntk_erf((x - mean) / c.sqrt(2 * variance))) #Neede to reverse the order of subtraction in x - mean so that the resulting shape (after broadcasting) is (mean.shape, x.shape)
    #return 0.5 * (1 + cntk_erf(-(mean - x) / c.sqrt(2 * variance)))    
    return 0.5 * (1 - cntk_erf((mean - x) / c.sqrt(2 * variance))) #same as above with 1 less negation

In [612]:
dtype=np.float64
numberOfQueries = 10
numberOfIntervals = 3
numberOfBoundaries = numberOfIntervals - 1
deltaMeans = c.parameter(shape=(numberOfQueries, 1), init=c.normal(1), dtype=dtype, name='deltaMeans')
deltaStDevs = c.parameter(shape=(numberOfQueries, 1), init=3, dtype=dtype, name='deltaStDevs')
#deltaStDevs = c.constant(shape=(numberOfQueries, 1), value=3, dtype=dtype, name='deltaStDevs')
deltaVariances = c.square(deltaStDevs)

initialInnerBoundaries = np.linspace(-1, 1, numberOfBoundaries, dtype=dtype)

#innerBoundaries = c.parameter(numberOfBoundaries, init=[-1, 1])
#innerBoundaries = c.parameter(numberOfBoundaries, init=(np.arange(numberOfBoundaries) / (numberOfBoundaries - 1) * 2 - 1)) #equally spaced boundaries in [-1, 1] range
innerBoundaries = c.parameter(numberOfBoundaries, init=initialInnerBoundaries, dtype=dtype, name='innerBoundaries')
#leftmostBoundary = c.constant(-math.inf, dtype=dtype, name='leftmostBoundary')
#rightmostBoundary = c.constant(+math.inf, dtype=dtype, name='rightmostBoundary')
#boundaries = c.splice(leftmostBoundary, innerBoundaries, rightmostBoundary, name='boundaries')

#vvv WTF? CNTK could not optimize the loss even though the input was not a constant but a parameter
#countsForIntervals = c.parameter(shape=(numberOfQueries, numberOfIntervals), init=np.random.randint(13, size=(numberOfQueries, numberOfIntervals)), dtype=np.int) #input
realDeltaMeans = np.random.normal(0, 3, size=(numberOfQueries))
countsForIntervalsInit = np.asarray([np.histogram(np.random.normal(mean, 1, size=50), bins=np.concatenate(([-np.inf], initialInnerBoundaries, [np.inf])))[0] for mean in realDeltaMeans])

countsForIntervals = c.constant(shape=(numberOfQueries, numberOfIntervals), value=countsForIntervalsInit, name='countsForIntervals') #input

In [613]:
#cntk_erf(boundaries).eval()

#cumulativeProbabilitiesForBoundaries = cntk_normal_cdf(boundaries, deltaMeans, deltaVariances) #Has NaN variance derivatives at infinities, so we add the infinite boundaries later
cumulativeProbabilitiesForInnerBoundaries = cntk_normal_cdf(innerBoundaries, deltaMeans, deltaVariances)
cumulativeProbabilitiesForBoundaries = c.splice(0, cumulativeProbabilitiesForInnerBoundaries, 1)

intervalProbabilities = c.slice(cumulativeProbabilitiesForBoundaries, 1, 1, 0) - c.slice(cumulativeProbabilitiesForBoundaries, 1, 0, -1) #P[i] = CDF[i]-CDF[i-1]

#intervalProbabilities.eval()

#(c.pow(intervalProbabilities, [[2,2,2],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1],[1,1,1]]  )).eval() #works!

occurrenceProbablilities = c.pow(intervalProbabilities, countsForIntervals, name='occurrenceProbablilities') #element_pow

totalProbability = c.reduce_prod(occurrenceProbablilities)
logTotalProbability = c.reduce_sum(c.log(occurrenceProbablilities))

#print((0 + countsForIntervals).eval())
#print(occurrenceProbablilities.eval())
#totalProbability.eval()


In [614]:
logTotalProbability.eval()

logTotalProbability.forward(None)[1]

df, fv = logTotalProbability.forward({}, [logTotalProbability.output], set([logTotalProbability.output]))
#print(df, fv)
value = list(fv.values())[0]
#print(value)
grad = logTotalProbability.backward(df, {logTotalProbability.output: np.ones_like(value)}, logTotalProbability.parameters)

print(grad)

{Parameter('deltaStDevs', [], [10 x 1]): array([[  7.15839658],
       [  0.77102241],
       [ -3.21799788],
       [  4.9576091 ],
       [  9.99648852],
       [ 20.06620402],
       [  9.59364466],
       [  5.28186027],
       [  8.05820313],
       [ -5.63429198]]), Parameter('deltaMeans', [], [10 x 1]): array([[ 17.49218012],
       [-13.06928597],
       [ -9.05971922],
       [-16.31672941],
       [ 18.47206956],
       [-21.65831963],
       [ 18.5051491 ],
       [-16.1920497 ],
       [ 16.71266962],
       [  1.70127679]]), Parameter('innerBoundaries', [], [2]): array([ 37.81882732, -34.40606858])}


In [615]:
dummy = c.input_variable(1, dtype=dtype)
#criterion = 1.0 - totalProbability + dummy
#criterion = 1.0 - occurrenceProbablilities + dummy
criterion = 1.0 - logTotalProbability + dummy

metric = c.reduce_mean(c.abs(c.reshape(deltaMeans, numberOfQueries) - realDeltaMeans))

criterion = criterion
lr_schedule = c.learning_rate_schedule(0.02, c.UnitType.minibatch)
learner = c.sgd(totalProbability.parameters, lr=lr_schedule)

progress_printer = c.logging.progress_print.ProgressPrinter(freq=100)
trainer = c.Trainer(criterion, (criterion, metric), [learner], [progress_printer])


#print((0+deltaMeans).eval())
#print(intervalProbabilities.eval())
#print(countsForIntervals.asarray())
#print(occurrenceProbablilities.eval())
#print(logTotalProbability.eval())

for i in range(0, 10000):
    trainer.train_minibatch({ dummy : np.asarray([0.0], dtype=np.float32) })
    if i % 100 == -1:
        #print((0+deltaMeans).eval())
        #print(occurrenceProbablilities.eval())
        print(intervalProbabilities.eval())

Learning rate per minibatch: 0.02


C:\local\Anaconda3-4.1.1-Windows-x86_64\envs\cntk-py35\lib\site-packages\cntk\core.py:351: UserWarning: your data is of type "float32", but your input variable (uid "Input14341") expects "<class 'numpy.float64'>". Please convert your data beforehand to speed up training.
  (sample.dtype, var.uid, str(var.dtype)))


 Minibatch[   1- 100]: loss = 210.401997 * 100, metric = 107.04% * 100;
 Minibatch[ 101- 200]: loss = 164.642073 * 100, metric = 88.71% * 100;
 Minibatch[ 201- 300]: loss = 164.379703 * 100, metric = 93.11% * 100;
 Minibatch[ 301- 400]: loss = 164.256477 * 100, metric = 96.09% * 100;
 Minibatch[ 401- 500]: loss = 164.188368 * 100, metric = 98.39% * 100;
 Minibatch[ 501- 600]: loss = 164.143764 * 100, metric = 100.28% * 100;
 Minibatch[ 601- 700]: loss = 164.111920 * 100, metric = 101.89% * 100;
 Minibatch[ 701- 800]: loss = 164.087800 * 100, metric = 103.30% * 100;
 Minibatch[ 801- 900]: loss = 164.068824 * 100, metric = 104.55% * 100;
 Minibatch[ 901-1000]: loss = 164.053465 * 100, metric = 105.67% * 100;
 Minibatch[1001-1100]: loss = 164.040750 * 100, metric = 106.70% * 100;
 Minibatch[1101-1200]: loss = 164.030026 * 100, metric = 107.64% * 100;
 Minibatch[1201-1300]: loss = 164.020850 * 100, metric = 108.51% * 100;
 Minibatch[1301-1400]: loss = 164.012901 * 100, metric = 109.33% * 1

In [616]:
df, fv = logTotalProbability.forward({}, [logTotalProbability.output], set([logTotalProbability.output]))
#print(df, fv)
value = list(fv.values())[0]
#print(value)
grad = logTotalProbability.backward(df, {logTotalProbability.output: np.ones_like(value)}, logTotalProbability.parameters, deltaStDevs)

print(grad)

{Parameter('deltaStDevs', [], [10 x 1]): array([[ -1.88951346e-04],
       [  4.66225008e-01],
       [  3.98200112e-01],
       [  1.91787472e-01],
       [  3.48015274e-01],
       [ -1.96455531e-04],
       [ -1.96348092e-04],
       [  3.63947376e-01],
       [ -2.15499869e-02],
       [ -2.53123782e-02]]), Parameter('deltaMeans', [], [10 x 1]): array([[  1.69928254e-05],
       [  4.47572193e-01],
       [  7.48336103e-01],
       [  7.79195311e-02],
       [ -1.73057320e-01],
       [ -1.90492883e-05],
       [  1.78409422e-05],
       [  2.36851712e-01],
       [ -8.88517874e-04],
       [  7.40245359e-02]]), Parameter('innerBoundaries', [], [2]): array([-1.60838051,  0.19760649])}


In [617]:
print('innerBoundaries=')
print(innerBoundaries.value)

print('realDeltaMeans=')
print(realDeltaMeans)

print('deltaMeans=')
print(deltaMeans.value)

print('countsForIntervalsInit=')
print(countsForIntervalsInit)

print((c.reshape(deltaMeans, numberOfQueries) - realDeltaMeans).eval())

innerBoundaries=
[-2.28512551  3.25542094]
realDeltaMeans=
[ 4.87303609 -1.83526924 -1.58451526 -3.21890587  2.59622289 -6.90461609
  5.23443529 -2.2836207   0.95711729 -0.74811113]
deltaMeans=
[[ 4.28245699]
 [-3.32880854]
 [-2.81699482]
 [-4.31803196]
 [ 4.34553693]
 [-5.36251144]
 [ 4.4167731 ]
 [-3.79286594]
 [ 3.25690975]
 [-1.90972831]]
countsForIntervalsInit=
[[ 0  0 50]
 [42  8  0]
 [35 15  0]
 [49  1  0]
 [ 0  2 48]
 [50  0  0]
 [ 0  0 50]
 [46  4  0]
 [ 2 23 25]
 [22 26  2]]
[-0.5905791  -1.4935393  -1.23247956 -1.0991261   1.74931405  1.54210465
 -0.8176622  -1.50924524  2.29979247 -1.16161718]
